In [0]:
%sql
SHOW TABLES;

In [0]:
%sql
use catalog `get_started`;
use schema `labuser`;

SHOW VOLUMES

In [0]:
%sql
LIST '/Volumes/get_started/labuser/myfiles';

### Create a Delta Table from a csv file

In [0]:
%sql
SELECT * FROM csv.`/Volumes/get_started/labuser/myfiles/employees.csv`;

In [0]:
%sql
SELECT * FROM text.`/Volumes/get_started/labuser/myfiles/employees.csv`;

In [0]:
%sql
SELECT * FROM read_files(
    '/Volumes/get_started/labuser/myfiles/employees.csv',
    format => 'csv',
    header => true,
    inferSchema => true
);

-- rescued data is provided by default

In [0]:
%sql
use catalog `get_started`;
use schema `labuser`;

-- drop table if exists current_employees
DROP TABLE IF EXISTS current_employees;
    
-- create a delta table
CREATE TABLE current_employees USING DELTA
AS
SELECT ID, Firstname, Country, Role 
FROM read_files(
    '/Volumes/get_started/labuser/myfiles/employees.csv',
    format => 'csv',
    header => true,
    inferSchema => true
);

In [0]:
# Get widget values
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

# Read a csv file and create Spark DataFrame
sdf = (spark.read
       .format("csv")
       .option("header", "true")
       .option("inferSchema", "true")
       .load(f'/Volumes/get_started/labuser/myfiles/employees.csv')
)
display(sdf)


# Create DELTA table from Spark DataFrame
# sdf.write.mode("overwrite").format("delta").saveAsTable(f"{catalog}.{schema}.current_employees_py")


In [0]:
%sql
SHOW TABLES

In [0]:
%sql
SELECT * FROM current_employees

In [0]:
%sql
DESCRIBE DETAIL current_employees

In [0]:
%sql
DESCRIBE EXTENDED current_employees

In [0]:
%sql 
DESCRIBE HISTORY current_employees

### INSERT, UPDATE, DELETE Records in DELTA TABLE

In [0]:
%sql
-- 1. Insert two employees
INSERT INTO current_employees
VALUES (1000, 'John', 'USA', 'Engineer'), (1001, 'Janet', 'Ghana', 'Manager');

-- 2. Update a record in table
UPDATE current_employees SET Country = 'England' WHERE ID = 322;

-- 3. Delete a record in table
DELETE FROM current_employees WHERE ID = 444;

In [0]:
%sql
SELECT * FROM current_employees ORDER BY ID;

In [0]:
%sql
DESCRIBE HISTORY current_employees;

### Using Time Travel

In [0]:
%sql
SELECT * FROM current_employees VERSION AS OF 2 
ORDER BY ID;

-- SELECT * FROM current_employees@v2;


In [0]:
%sql
SELECT * FROM current_employees@v0;


In [0]:
%sql
DROP TABLE IF EXISTS current_employees;
DROP TABLE IF EXISTS current_employees_py;